In [ ]:
import librosa
import numpy as np
import tensorflow as tf

def load_and_pad(file_path, sr=16000):
    try:
        y, _ = librosa.load(file_path, sr=sr)
        if len(y) < sr:
            y = np.pad(y, (0, sr - len(y)))
        return y
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def preprocess_single_file_vgg(file_path, sr=16000, n_mfcc=40):
    y = load_and_pad(file_path, sr=sr)
    if y is None:
        raise ValueError("Audio loading failed")

    # Extract MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc = (mfcc - np.mean(mfcc)) / (np.std(mfcc) + 1e-8)

    # Pad or trim MFCC to (40, 64)
    if mfcc.shape[1] < 64:
        pad_width = 64 - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)))
    else:
        mfcc = mfcc[:, :64]

    # Resize to (64, 64) and convert to 3-channel
    mfcc_resized = tf.image.resize(mfcc[..., np.newaxis], [40, 64]).numpy()
    mfcc_rgb = np.repeat(mfcc_resized, 1, axis=-1)  # shape (64, 64, 3)
    mfcc_rgb = np.expand_dims(mfcc_rgb, axis=0)     # shape (1, 64, 64, 3)

    return mfcc_rgb


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load VGG16 model
model = load_model(r"C:\Users\iksmh\Downloads\VGG\vgg16_full_86.h5")

# Predict on your sample
raw_file = r"C:\Users\iksmh\Downloads\test_audio\test_audio\Fake.wav"  
processed_input = preprocess_single_file_vgg(raw_file)
prediction = model.predict(processed_input)

# Output result
predicted_class = np.argmax(prediction)
class_names = ['real', 'fake']
print(f"Predicted Class: {class_names[predicted_class]}, Confidence: {prediction[0][predicted_class]:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Predicted Class: real, Confidence: 0.9991
